In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

NumberOfRows = 10000

filePath = '../Downloads/'
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score
import scipy.sparse as sp

In [3]:
businuessColumns = ['state', 'business_id'] #'review_count', 'stars', 
userColumns = ["user_id", "useful", "fans", "funny", "cool", "average_stars", "review_count"] #, "elite", "compliment_photos", "compliment_list", "compliment_funny", "compliment_plain", "compliment_note", "compliment_writer", "compliment_cute", "compliment_more", "compliment_hot", "compliment_profile", "compliment_cool",
ratingsColumns = ["business_id", "stars", "user_id"] #"cool", "funny", "useful", 
targetColumn = "stars"

In [4]:
business = pd.read_csv(filePath + "business_cleaned.csv", usecols=businuessColumns)
user = pd.read_csv(filePath + 'user_cleaned.csv', usecols=userColumns)
ratings = pd.read_csv(filePath + 'yelp_ratings.csv', usecols=ratingsColumns, nrows=NumberOfRows)

In [5]:
print(business['business_id'].nunique())
print(business['business_id'].drop_duplicates().shape[0])

print(user['user_id'].nunique())
print(user['user_id'].drop_duplicates().shape[0])

158525
158525
818922
818922


In [6]:
ratings = pd.merge(ratings, business, how='left', left_on='business_id', right_on='business_id')
ratings = pd.merge(ratings, user, how='left', left_on='user_id', right_on='user_id')

In [7]:
ratings['business_id'] = ratings['business_id'].astype('category')
ratings['business_id'] = ratings['business_id'].cat.codes

ratings['user_id'] = ratings['user_id'].astype('category')
ratings['user_id'] = ratings['user_id'].cat.codes

In [8]:
def convertDummies(df, column_name):
    df = pd.concat([
        df.drop(column_name, axis=1),
        pd.get_dummies(df[column_name], prefix=column_name+'_')
    ], axis=1)
    print(column_name, ' is ready.')
    return df

In [9]:
ratings = convertDummies(ratings, 'business_id')
ratings = convertDummies(ratings, 'user_id')
ratings = convertDummies(ratings, 'state')

business_id  is ready.
user_id  is ready.
state  is ready.


In [10]:
train_X = ratings.drop(targetColumn, axis=1).iloc[:-2000]
train_y = ratings[targetColumn].iloc[:-2000]

test_X = ratings.drop(targetColumn, axis=1).iloc[-2000:]
test_y = ratings[targetColumn].iloc[-2000:]

In [11]:
tr_X = train_X.to_numpy()
tr_y = train_y.to_numpy()
te_X = test_X.to_numpy()
te_y = test_y.to_numpy()
X_tr_sparse = sp.csr_matrix(tr_X)
X_te_sparse = sp.csr_matrix(te_X)

In [25]:
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error

model = TFFMRegressor(
    order=3, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=50, 
    batch_size=800,
    init_std=0.001,
    input_type='sparse',
)
model.fit(X_tr_sparse, tr_y, show_progress=True)

100%|██████████| 50/50 [00:03<00:00, 14.32epoch/s]


In [26]:
predictions = model.predict(X_te_sparse )
from sklearn.metrics import mean_squared_error
print('[order={}] accuracy: {}'.format(3, accuracy_score(te_y, predictions > 0)))
print(mean_squared_error(te_y,predictions))

[order=3] accuracy: 0.0395
147876.61413434893


In [27]:
predictions

array([-3.3405628 , -0.77704525,  0.0748506 , ..., -0.8060788 ,
       -0.04460888, -0.5346799 ], dtype=float32)

In [28]:
predictions*10

array([-33.40563   ,  -7.7704525 ,   0.74850595, ...,  -8.060788  ,
        -0.4460888 ,  -5.346799  ], dtype=float32)

In [29]:
tr_y

array([5., 2., 1., ..., 2., 5., 4.])